In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [14]:
df = pd.read_csv('TRAIN.csv')
test = pd.read_csv('TEST.csv')

In [15]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
x.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Order
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62


In [16]:
x = x.drop(columns = ['ID', "Order"])

# Preprocessing 

In [17]:
x = pd.get_dummies(x, columns = ['Location_Type', 'Store_Type', 'Region_Code'])
x

,Store_id,Date,Holiday,Discount,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,1,2018-01-01,1,Yes,0,0,1,0,0,1,0,0,0,1,0,0,0
1,253,2018-01-01,1,Yes,0,1,0,0,0,0,0,0,1,1,0,0,0
2,252,2018-01-01,1,Yes,0,1,0,0,0,0,0,1,0,1,0,0,0
3,251,2018-01-01,1,Yes,0,0,1,0,0,0,1,0,0,1,0,0,0
4,250,2018-01-01,1,Yes,0,0,1,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,149,2019-05-31,1,Yes,0,0,1,0,0,0,1,0,0,0,1,0,0
188336,153,2019-05-31,1,No,0,1,0,0,0,0,0,0,1,1,0,0,0
188337,154,2019-05-31,1,No,0,0,1,0,0,1,0,0,0,0,1,0,0
188338,155,2019-05-31,1,Yes,1,0,0,0,0,0,0,1,0,0,1,0,0


In [18]:
le = LabelEncoder()
x['Discount'] = le.fit_transform(x['Discount'])

In [19]:
x['Date'] = pd.to_datetime(x['Date'], format = '%Y-%m-%d', errors = 'coerce')

In [20]:
x['Date'] = x['Date'].dt.dayofweek

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [21]:
x

,Store_id,Date,Holiday,Discount,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,1,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0
1,253,0,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0
2,252,0,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0
3,251,0,1,1,0,0,1,0,0,0,1,0,0,1,0,0,0
4,250,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,149,4,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0
188336,153,4,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0
188337,154,4,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0
188338,155,4,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0


In [34]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               2304      
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 129       
Total params: 199,937
Trainable params: 199,937
Non-trainable params: 0
________________________________________________

In [35]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [37]:
NN_model.fit(x, y, epochs=100, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/100
4707/4709 [============================>.] - ETA: 0s - loss: 8102.6479 - mean_absolute_error: 8102.6479
Epoch 00001: val_loss improved from 9306.15625 to 9232.57617, saving model to Weights-001--9232.57617.hdf5
4709/4709 [==============================] - 14s 3ms/step - loss: 8102.5840 - mean_absolute_error: 8102.5840 - val_loss: 9232.5762 - val_mean_absolute_error: 9232.5762
Epoch 2/100
4703/4709 [============================>.] - ETA: 0s - loss: 8004.2720 - mean_absolute_error: 8004.2720
Epoch 00002: val_loss improved from 9232.57617 to 8963.78809, saving model to Weights-002--8963.78809.hdf5
4709/4709 [==============================] - 15s 3ms/step - loss: 8004.4556 - mean_absolute_error: 8004.4556 - val_loss: 8963.7881 - val_mean_absolute_error: 8963.7881
Epoch 3/100
4697/4709 [============================>.] - ETA: 0s - loss: 7959.3169 - mean_absolute_error: 7959.3169
Epoch 00003: val_loss did not improve from 8963.78809
4709/4709 [==============================] - 15s

Epoch 24/100
4703/4709 [============================>.] - ETA: 0s - loss: 7635.8359 - mean_absolute_error: 7635.8359
Epoch 00024: val_loss did not improve from 8865.97461
4709/4709 [==============================] - 15s 3ms/step - loss: 7635.9326 - mean_absolute_error: 7635.9326 - val_loss: 8935.7715 - val_mean_absolute_error: 8935.7715
Epoch 25/100
4698/4709 [============================>.] - ETA: 0s - loss: 7631.9849 - mean_absolute_error: 7631.9849
Epoch 00025: val_loss did not improve from 8865.97461
4709/4709 [==============================] - 16s 3ms/step - loss: 7630.3120 - mean_absolute_error: 7630.3120 - val_loss: 8868.2354 - val_mean_absolute_error: 8868.2354
Epoch 26/100
4709/4709 [==============================] - ETA: 0s - loss: 7621.1006 - mean_absolute_error: 7621.1006
Epoch 00026: val_loss did not improve from 8865.97461
4709/4709 [==============================] - 16s 3ms/step - loss: 7621.1006 - mean_absolute_error: 7621.1006 - val_loss: 8930.7969 - val_mean_absolute_

Epoch 48/100
4701/4709 [============================>.] - ETA: 0s - loss: 7472.7583 - mean_absolute_error: 7472.7583
Epoch 00048: val_loss did not improve from 8751.38281
4709/4709 [==============================] - 14s 3ms/step - loss: 7471.8945 - mean_absolute_error: 7471.8945 - val_loss: 8762.9570 - val_mean_absolute_error: 8762.9570
Epoch 49/100
4709/4709 [==============================] - ETA: 0s - loss: 7457.8696 - mean_absolute_error: 7457.8696
Epoch 00049: val_loss did not improve from 8751.38281
4709/4709 [==============================] - 15s 3ms/step - loss: 7457.8696 - mean_absolute_error: 7457.8696 - val_loss: 8801.9756 - val_mean_absolute_error: 8801.9756
Epoch 50/100
4706/4709 [============================>.] - ETA: 0s - loss: 7454.7080 - mean_absolute_error: 7454.7080
Epoch 00050: val_loss did not improve from 8751.38281
4709/4709 [==============================] - 14s 3ms/step - loss: 7455.1719 - mean_absolute_error: 7455.1719 - val_loss: 8869.8174 - val_mean_absolute_

Epoch 72/100
4705/4709 [============================>.] - ETA: 0s - loss: 7355.5708 - mean_absolute_error: 7355.5708
Epoch 00072: val_loss did not improve from 8706.64453
4709/4709 [==============================] - 13s 3ms/step - loss: 7355.8442 - mean_absolute_error: 7355.8442 - val_loss: 8843.6826 - val_mean_absolute_error: 8843.6826
Epoch 73/100
4692/4709 [============================>.] - ETA: 0s - loss: 7353.5884 - mean_absolute_error: 7353.5884
Epoch 00073: val_loss did not improve from 8706.64453
4709/4709 [==============================] - 13s 3ms/step - loss: 7353.2261 - mean_absolute_error: 7353.2261 - val_loss: 8906.0068 - val_mean_absolute_error: 8906.0068
Epoch 74/100
4706/4709 [============================>.] - ETA: 0s - loss: 7353.5444 - mean_absolute_error: 7353.5444
Epoch 00074: val_loss did not improve from 8706.64453
4709/4709 [==============================] - 13s 3ms/step - loss: 7353.2827 - mean_absolute_error: 7353.2827 - val_loss: 8850.3408 - val_mean_absolute_

Epoch 96/100
4700/4709 [============================>.] - ETA: 0s - loss: 7301.2222 - mean_absolute_error: 7301.2222
Epoch 00096: val_loss did not improve from 8706.64453
4709/4709 [==============================] - 16s 3ms/step - loss: 7301.6060 - mean_absolute_error: 7301.6060 - val_loss: 8852.4141 - val_mean_absolute_error: 8852.4141
Epoch 97/100
4704/4709 [============================>.] - ETA: 0s - loss: 7298.2432 - mean_absolute_error: 7298.2432- ETA: 1s - loss: 7296.0376 - mean_ab
Epoch 00097: val_loss did not improve from 8706.64453
4709/4709 [==============================] - 17s 4ms/step - loss: 7297.8740 - mean_absolute_error: 7297.8740 - val_loss: 8816.6182 - val_mean_absolute_error: 8816.6182
Epoch 98/100
4704/4709 [============================>.] - ETA: 0s - loss: 7298.0410 - mean_absolute_error: 7298.0410
Epoch 00098: val_loss did not improve from 8706.64453
4709/4709 [==============================] - 16s 3ms/step - loss: 7297.7339 - mean_absolute_error: 7297.7339 - val

In [26]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [27]:
original_test = test

In [28]:
test = test.drop(columns = ['ID'])
test = pd.get_dummies(test, columns = ['Location_Type', 'Store_Type', 'Region_Code'])
test

,Store_id,Date,Holiday,Discount,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,171,2019-06-01,0,No,0,1,0,0,0,0,0,0,1,0,0,1,0
1,172,2019-06-01,0,No,1,0,0,0,0,1,0,0,0,1,0,0,0
2,173,2019-06-01,0,No,0,1,0,0,0,0,0,0,1,1,0,0,0
3,174,2019-06-01,0,No,1,0,0,0,0,1,0,0,0,0,0,0,1
4,170,2019-06-01,0,No,1,0,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,186,2019-07-31,0,No,0,0,0,0,1,0,1,0,0,0,1,0,0
22261,11,2019-07-31,0,No,0,1,0,0,0,0,0,0,1,1,0,0,0
22262,185,2019-07-31,0,Yes,1,0,0,0,0,1,0,0,0,0,0,1,0
22263,69,2019-07-31,0,No,1,0,0,0,0,1,0,0,0,0,0,0,1


In [29]:
test['Discount'] = le.fit_transform(test['Discount'])
test

,Store_id,Date,Holiday,Discount,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,171,2019-06-01,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0
1,172,2019-06-01,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0
2,173,2019-06-01,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
3,174,2019-06-01,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
4,170,2019-06-01,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,186,2019-07-31,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0
22261,11,2019-07-31,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
22262,185,2019-07-31,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0
22263,69,2019-07-31,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1


In [30]:
test['Date'] = pd.to_datetime(test['Date'], format = '%Y-%m-%d', errors = 'coerce')
test['Date'] = test['Date'].dt.dayofweek
test

,Store_id,Date,Holiday,Discount,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,171,5,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0
1,172,5,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0
2,173,5,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
3,174,5,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
4,170,5,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,186,2,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0
22261,11,2,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0
22262,185,2,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0
22263,69,2,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1


In [31]:
pred = NN_model.predict(test)

In [32]:
pred

array([[58928.465],
       [37938.18 ],
       [86480.21 ],
       ...,
       [45462.   ],
       [31912.982],
       [25879.916]], dtype=float32)

In [33]:
submit = pd.DataFrame()
submit['ID'] = original_test['ID']
submit['Sales'] = pred
submit.to_csv('Submission8.csv', index = False)
print("file saved")

file saved
